<a href="https://colab.research.google.com/github/Ms-Noxolo/Team_EN3_Jozi/blob/master/Collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!pip install scikit-surprise

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Importing modules
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import math
from surprise import Dataset, Reader, SVD, KNNWithMeans
from surprise.model_selection import train_test_split

In [3]:
# Importing data
train = pd.read_csv('/content/drive/My Drive/Music/train.csv')
test = pd.read_csv('/content/drive/My Drive/Music/test.csv')
movies = pd.read_csv('/content/drive/My Drive/Music/movies.csv')

In [4]:
train.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [5]:
test.head()

,userId,movieId
0,1,2011
1,1,4144
2,1,5767
3,1,6711
4,1,7318


In [6]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
#Dropping timestamp
train.drop('timestamp',axis=1,inplace=True)

In [8]:
#Reducing the data to can only use the sample
X = train[:1000000]

In [9]:
#Loading data and determining the range of the ratings
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(X, reader)
trainset, testset = train_test_split(data, test_size=0.25)

In [10]:
#Training the model
model = SVD(n_factors=150,random_state=42)
model.fit(trainset)

In [11]:
# making the ratings predictions and comparing them with the real values
c = []
for a,i,j in testset:
    c.append(round(model.predict(i,j).est))
a = []
for i,j,k in testset:
    a.append(k)
math.sqrt(mean_squared_error(a,c))

1.3130357953993486

In [12]:
#Creating a new dataframe of movieId and titles
new = movies.loc[:,['movieId','title']]

In [13]:
#Merging data with titles to can create a pivot_table
X = pd.merge(X,new,on = 'movieId')

In [14]:
X

,userId,movieId,rating,title
0,5163,57669,4.0,In Bruges (2008)
1,87388,57669,3.5,In Bruges (2008)
2,137050,57669,4.0,In Bruges (2008)
3,120490,57669,4.5,In Bruges (2008)
4,50616,57669,4.5,In Bruges (2008)
...,...,...,...,...
999995,60438,105079,0.5,His Regeneration (1915)
999996,154044,148480,3.0,Double Down (2005)
999997,111324,110106,2.5,Naked Harbour (Vuosaari) (2012)
999998,72315,173345,3.5,Cat City (1986)


In [15]:
#creating a pivot table to can compare the movies to can recommend
table = X[:800000].pivot_table(index=['userId'], columns = ['title'],values = 'rating').fillna(0)

In [16]:
table.head()

title,(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Rent Boys (2000),102 Dalmatians (2000),12 Angry Men (1957),12 Years a Slave (2013),12:08 East of Bucharest (A fost sau n-a fost?) (2006),13 Going on 30 (2004),13th (2016),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),17 Again (2009),1984 (1956),1991: The Year Punk Broke (1992),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),21 Grams (2003),21 Jump Street (2012),22 Jump Street (2014),25th Hour (2002),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),3 Idiots (2009),3 Ninjas (1992),3 Ninjas Kick Back (1994),30 Minutes or Less (2011),300 (2007),...,Year of the Dragon (1985),"Yearling, The (1946)",Yellow Submarine (1968),Yes Man (2008),Yojimbo (1961),You Can Count on Me (2000),You Don't Mess with the Zohan (2008),You Kill Me (2007),You Only Live Once (1937),You've Got Mail (1998),"You, Me and Dupree (2006)",Young Einstein (1988),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Youngblood (1986),Your Friends and Neighbors (1998),Your Highness (2011),Your Name. (2016),Youth in Revolt (2009),ZMD: Zombies of Mass Destruction (2009),Zack and Miri Make a Porno (2008),Zardoz (1974),Zeitgeist: Addendum (2008),Zelig (1983),Zero Effect (1998),Zero Kelvin (Kjærlighetens kjøtere) (1995),Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zodiac (2007),Zombeavers (2014),Zombieland (2009),Zone Troopers (1985),Zoolander (2001),Zootopia (2016),[REC] (2007),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#creating a similarity table to can compare the distance between movies using pearson correlation
similarity_df = table.corr(method='pearson')

Given that the user has seen a movie and rated it, we are going to recommend him another movie using the rating that he gave to th movie

In [ ]:
#Creating a function that sort movies according to their similarity score and return a dataframe of movies he might like
def find_similar(movie_name,rating):
    score = similarity_df[movie_name]*(rating-2.5)
    score = score.sort_values(ascending = False)
    similar_movies = pd.DataFrame()
    for movie,rating in movies:
        similar_movies = similar_movies.append(find_similar(movie,rating),ignore_index=True)
    similar_movies.head(10)
    Y = pd.DataFrame(similar_movies.sum().sort_values(ascending=False).head(10)).drop(0,axis=1)
    
    return Y

In [ ]:
#Finally recommending a new movie
find_similar((500) Days of Summer (2009),5)